In [2]:
from atproto import FirehoseSubscribeReposClient, parse_subscribe_repos_message, models, CAR

In [3]:
client = FirehoseSubscribeReposClient()

In [5]:
def on_message_handler(message):
    commit = parse_subscribe_repos_message(message)
    if not isinstance(commit, models.ComAtprotoSyncSubscribeRepos.Commit):
        return
    
    if not commit.blocks:
        return
    
    car = CAR.from_bytes(commit.blocks)
    for op in commit.ops:
        if op.action in ["create"] and op.cid:
            data = car.blocks.get(op.cid)

            if data['$type'] == 'app.bsky.feed.post':
                text = data['text']

                if 'coffee' in text:
                    print(text)


client.start(on_message_handler)

Mysteries for Winter weather...
Pour a cup of coffee and read📚
Agatha Christie. Midwinter murder, fireside tales from the queen of mystery
Louise Penny. A fatal grace
Lucy Foley. The hunting party
coffee goes so well with a good blunt 🤤 

good morning mootsss!! 🐄
A damn fine cup of coffee
I saw this posted elsewhere (credit by a Moroccan artist).  The message’s intention is that through all of our years our hearts remain young.  I also interpret it to mean good coffee connections build and endure.  #coffeeconnections #goodconversation #throughtheyears
bwew huu .  I take another sip of coffee.  that will show them.
I just read a share about putting chocolates into coffee. I am thankful.☕🍫 The app went to quickly for me to directly say it.
It's Saturday. Don't forget your coffee ☕
I can't take him to coffee shops anymore...
Who would win in a battle between: a coffee cup and a banana
I'm sitting here, waiting for my caffeine to kick in.
I fucking forgot my coffee at the coffee pot.
Think

KeyboardInterrupt: 